In [1]:
import os
from typing import List
from tqdm import tqdm
from numba import jit
from numba.typed import List as NumbaList
import numpy as np
import optuna
import joblib
import random
import gc
gc.enable()

from src.utils import TravelerMap
from src.fitness import TravelerFitness
from src.generator import TravelGenerator
from src.selector import TravelerSelector
from src.mutation import TravelMutation
from src.crossover import TravelerCrossover

DATA_DIR = './data'
MAP_PATHS = os.listdir(DATA_DIR)

In [2]:
#@jit(nopython=True)
def evaluate(distances_matrix: List[List[float]], nodes_amount: int, 
             pop_size: int, iters: int, trials: int, 
             cross_pairs: int, mut_mode: int, mut_opt_prob: float, 
             mut_cand_percent: float, ff_age_limit: int = 500) -> List[float]:

    best_fitnesses, best_iters, best_solutions = [], [], []
    for _ in range(trials):
        # Инициализация операторов эволюционного процесса
        generator = TravelGenerator(nodes_amount)
        ff = TravelerFitness(generator.dim, distances_matrix)
        selection_oprt = TravelerSelector(pop_size, generator.dim)        
        crossover_oprt = TravelerCrossover(generator.dim, cross_pairs)
        mutate_oprt = TravelMutation(generator.dim, mut_mode, mut_opt_prob, 
                                     mut_cand_percent)
        
        #print("Инициализация популяции")
        base_pop = generator.get_population(pop_size)

        #print("Старт эволюционного процесса...")
        cur_fitness, flag = [], False
        process = tqdm(range(iters))
        for iter_idx in process:

            #          
            if ff.result_age > ff_age_limit:
                flag = True
                break
    
            #
            #print(f"Мутация {base_pop.shape} {base_pop.dtype} кандидатов...")
            old_pop = mutate_oprt.apply(base_pop)
            old_fit = np.array([ff.calculate_fitness(sol) for sol in old_pop], dtype=np.int32)

            #print(f"Кроссовер {old_pop.shape} {old_pop.dtype} кандидатов...")
            new_pop = crossover_oprt.mate(old_pop)
            new_fit = np.array([ff.calculate_fitness(sol) for sol in new_pop], dtype=np.int32)

            union_pop = np.concatenate((old_pop, new_pop))
            union_fit = np.concatenate((old_fit, new_fit))
                
            #print(f"Селекция {union_pop.shape} {union_pop.dtype} кандидатов...")
            base_pop = selection_oprt.filter_population(union_pop, union_fit)

            #cur_fitness.append(ff.best_result)
            process.set_postfix({'ff': ff.best_result})
            ff.result_age += 1

        best_fitnesses.append(ff.best_result)
        best_iters.append( iter_idx+1-ff_age_limit if flag else iter_idx+1 )
        best_solutions.append(np.copy(ff.best_solution))

        #
        del ff
        del selection_oprt
        del crossover_oprt
        del mutate_oprt
        del generator

        del base_pop
        del union_pop
        del union_fit
        del new_pop
        del new_fit
        del old_pop
        del old_fit

        gc.collect()
     
    return best_fitnesses, best_iters, best_solutions

In [4]:
def objective(trial):
    global MAP_IDX
    global EVALALG_ITERATIONS
    global EVALALG_TRIALS
    global POPULATION_SIZE
    
    gc.collect()

    map_obj = TravelerMap()
    map_obj.load_map(f"{DATA_DIR}/{MAP_PATHS[MAP_IDX]}")

    cross_pairs = trial.suggest_int('cross_pairs', 160, 240, step=40)
    mut_mode = trial.suggest_categorical('mut_mode', [0, 1])
    mut_opt_prob = trial.suggest_float("mut_opt_prob", 0, 1, step=0.2)
    mut_cand_percent = trial.suggest_float("mut_opt_percent", 0.6, 1.0, step=0.2)

    #
    best_fitnesses, best_iters, _ = evaluate(map_obj.distances, map_obj.nodes_amount, POPULATION_SIZE, 
                                             EVALALG_ITERATIONS, EVALALG_TRIALS, cross_pairs, 
                                             mut_mode, mut_opt_prob, mut_cand_percent)

    mean_fitness = np.mean(best_fitnesses)
    #mean_iters = np.mean(best_iters)

    return mean_fitness

In [5]:
for i, map_file in enumerate(MAP_PATHS):

    MAP_IDX = i
    OPTUNA_TRIALS = 20
    EVALALG_TRIALS = 3
    EVALALG_ITERATIONS = 2000
    POPULATION_SIZE = 200

    print(MAP_IDX, EVALALG_ITERATIONS, EVALALG_TRIALS, map_file)

    study = optuna.create_study(directions=['minimize'])
    study.optimize(objective, n_trials=OPTUNA_TRIALS, show_progress_bar=True, gc_after_trial=True)

    joblib.dump(study, f"./logs/{map_file.split('.')[0]}_optuna_study.pkl")

[I 2024-05-25 05:15:54,790] A new study created in memory with name: no-name-c8400062-2d8c-4b1c-8f9e-442509eefea0


0 2000 3 pma343.tsp


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2024-05-25 05:19:27,525] Trial 0 finished with value: 2273.793701171875 and parameters: {'cross_pairs': 160, 'mut_mode': 0, 'mut_opt_prob': 0.4, 'mut_opt_percent': 0.8}. Best is trial 0 with value: 2273.793701171875.
[I 2024-05-25 05:23:42,846] Trial 1 finished with value: 1901.6880289713542 and parameters: {'cross_pairs': 200, 'mut_mode': 0, 'mut_opt_prob': 0.0, 'mut_opt_percent': 1.0}. Best is trial 1 with value: 1901.6880289713542.
[I 2024-05-25 05:28:50,612] Trial 2 finished with value: 2102.710693359375 and parameters: {'cross_pairs': 240, 'mut_mode': 0, 'mut_opt_prob': 0.6000000000000001, 'mut_opt_percent': 1.0}. Best is trial 1 with value: 1901.6880289713542.
[I 2024-05-25 05:31:39,190] Trial 3 finished with value: 11064.2158203125 and parameters: {'cross_pairs': 160, 'mut_mode': 1, 'mut_opt_prob': 0.2, 'mut_opt_percent': 0.6}. Best is trial 1 with value: 1901.6880289713542.
[I 2024-05-25 05:34:21,678] Trial 4 finished with value: 14862.788736979166 and parameters: {'cross_pa

[I 2024-05-25 06:37:13,311] A new study created in memory with name: no-name-45b83f63-6bc2-4f1b-abba-30e75942acd9


...

[I 2024-05-25 06:38:17,962] Trial 0 finished with value: 5924.100911458333 and parameters: {'cross_pairs': 160, 'mut_mode': 1, 'mut_opt_prob': 0.6000000000000001, 'mut_opt_percent': 1.0}. Best is trial 0 with value: 5924.100911458333.
[I 2024-05-25 06:40:33,815] Trial 1 finished with value: 1252.134033203125 and parameters: {'cross_pairs': 200, 'mut_mode': 0, 'mut_opt_prob': 0.2, 'mut_opt_percent': 0.6}. Best is trial 1 with value: 1252.134033203125.
[I 2024-05-25 06:42:32,430] Trial 2 finished with value: 4303.054117838542 and parameters: {'cross_pairs': 240, 'mut_mode': 1, 'mut_opt_prob': 0.4, 'mut_opt_percent': 0.6}. Best is trial 1 with value: 1252.134033203125.
[I 2024-05-25 06:43:52,269] Trial 3 finished with value: 6267.443684895833 and parameters: {'cross_pairs': 160, 'mut_mode': 1, 'mut_opt_prob': 1.0, 'mut_opt_percent': 0.6}. Best is trial 1 with value: 1252.134033203125.
[I 2024-05-25 06:45:38,783] Trial 4 finished with value: 4087.683837890625 and parameters: {'cross_pairs'

[I 2024-05-25 07:19:28,515] A new study created in memory with name: no-name-09c3c5db-d453-4c92-a3fc-bf85575e01ea


[I 2024-05-25 07:19:28,455] Trial 19 finished with value: 1176.03369140625 and parameters: {'cross_pairs': 240, 'mut_mode': 0, 'mut_opt_prob': 0.0, 'mut_opt_percent': 1.0}. Best is trial 19 with value: 1176.03369140625.
2 2000 3 xqf131.tsp


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2024-05-25 07:20:00,992] Trial 0 finished with value: 1692.948974609375 and parameters: {'cross_pairs': 160, 'mut_mode': 1, 'mut_opt_prob': 0.6000000000000001, 'mut_opt_percent': 0.6}. Best is trial 0 with value: 1692.948974609375.
[I 2024-05-25 07:20:52,039] Trial 1 finished with value: 1084.2966715494792 and parameters: {'cross_pairs': 160, 'mut_mode': 0, 'mut_opt_prob': 1.0, 'mut_opt_percent': 1.0}. Best is trial 1 with value: 1084.2966715494792.
[I 2024-05-25 07:21:38,955] Trial 2 finished with value: 1599.9923502604167 and parameters: {'cross_pairs': 200, 'mut_mode': 1, 'mut_opt_prob': 0.8, 'mut_opt_percent': 0.6}. Best is trial 1 with value: 1084.2966715494792.
[I 2024-05-25 07:22:42,944] Trial 3 finished with value: 647.1796671549479 and parameters: {'cross_pairs': 240, 'mut_mode': 1, 'mut_opt_prob': 0.0, 'mut_opt_percent': 0.8}. Best is trial 3 with value: 647.1796671549479.
[I 2024-05-25 07:23:36,577] Trial 4 finished with value: 675.25 and parameters: {'cross_pairs': 200, 

In [12]:
MAP_IDX = 2
study = joblib.load(f"./logs/{MAP_PATHS[MAP_IDX].split('.')[0]}_optuna_study.pkl")
PARAMS = study.best_trials[0].params

map_obj = TravelerMap()
map_obj.load_map(f"{DATA_DIR}/{MAP_PATHS[MAP_IDX]}")

best_fitnesses, best_iters, best_solutions = evaluate(map_obj.distances, map_obj.nodes_amount, 
                                                      200, 4000, 10, PARAMS['cross_pairs'], 
                                                      PARAMS['mut_mode'], PARAMS['mut_opt_prob'], 
                                                      PARAMS['mut_opt_percent'])
mean_fitness = np.mean(best_fitnesses)
mean_iters = np.mean(best_iters)
print(mean_fitness, mean_iters)
#map_obj.plot_solution(best_solutions[0])

 62%|██████▏   | 2461/4000 [00:23<00:14, 103.29it/s, ff=611]   


632.4722961425781 2215.4


In [9]:
mean_iters = np.mean(best_iters)

In [10]:
mean_iters

4000.0